# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
os.chdir("../")   # change directory to location of api_key

from pprint import pprint
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame

In [2]:
# read cities from Watcsv and place in data frome

cityDataDF = pd.read_csv("Weatherpy/output_data/cities.csv")
cityDataDF.head()



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Aklavik,68.2191,-135.0107,64.58,66.0,100.0,4.36,CA,1.660790e+09
1,1,Paamiut,61.9940,-49.6678,41.77,88.0,17.0,12.30,GL,1.660790e+09
2,2,Ust-Tsilma,65.4410,52.1498,53.98,75.0,24.0,4.79,RU,1.660790e+09
3,3,viligili,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,illoqqortoormiut,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
len(cityDataDF)

613

In [4]:
cleancityDataDF = cityDataDF.dropna()
cleancityDataDF


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Aklavik,68.2191,-135.0107,64.58,66.0,100.0,4.36,CA,1.660790e+09
1,1,Paamiut,61.9940,-49.6678,41.77,88.0,17.0,12.30,GL,1.660790e+09
2,2,Ust-Tsilma,65.4410,52.1498,53.98,75.0,24.0,4.79,RU,1.660790e+09
5,5,Preobrazheniye,42.9019,133.9064,75.00,65.0,2.0,5.88,RU,1.660790e+09
6,6,Hobart,-42.8794,147.3294,60.26,75.0,75.0,23.02,AU,1.660790e+09
...,...,...,...,...,...,...,...,...,...,...
608,608,Taoudenni,22.6783,-3.9836,88.84,46.0,65.0,14.88,ML,1.660791e+09
609,609,Belaya Gora,68.5333,146.4167,57.33,69.0,99.0,8.37,RU,1.660791e+09
610,610,Burnie,-41.0667,145.9167,53.94,88.0,93.0,22.06,AU,1.660791e+09
611,611,Ciudad Bolivar,8.1222,-63.5497,80.17,68.0,23.0,11.25,VE,1.660791e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#configure heat map and read key
gmaps.configure(api_key=g_key)

In [6]:
#heat map of humidity 
locations=cleancityDataDF[["Lat","Lng"]]
humidity=cleancityDataDF["Humidity"]
fig=gmaps.figure()
#fig


#add heat map layer to figure 
heatLayer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heatLayer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria


In [7]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.

#Zero cloudiness.
#Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

weatherConditionDF = cleancityDataDF[(cleancityDataDF["Max Temp"] > 70 )& 
                                         (cleancityDataDF["Max Temp"]  < 80) &  
                                         (cleancityDataDF["Wind Speed"] < 10) & 
                                         (cleancityDataDF["Cloudiness"] == 0)]
weatherConditionDF.head()



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
123,123,Yangi Marg`ilon,40.4272,71.7189,75.92,35.0,0.0,4.61,UZ,1.660790e+09
145,145,Ürümqi,43.8010,87.6005,70.75,27.0,0.0,2.24,CN,1.660790e+09
158,158,San Javier,37.8063,-0.8374,76.98,74.0,0.0,4.00,ES,1.660790e+09
162,162,Constantine,36.3650,6.6147,78.21,38.0,0.0,4.61,DZ,1.660790e+09
223,223,Termez,37.2242,67.2783,77.09,38.0,0.0,0.00,UZ,1.660790e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = weatherConditionDF.copy()
#hotel_df= hotel_df[["City","Country","Lat","Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
123,123,Yangi Marg`ilon,40.4272,71.7189,75.92,35.0,0.0,4.61,UZ,1.660790e+09,
145,145,Ürümqi,43.8010,87.6005,70.75,27.0,0.0,2.24,CN,1.660790e+09,
158,158,San Javier,37.8063,-0.8374,76.98,74.0,0.0,4.00,ES,1.660790e+09,
162,162,Constantine,36.3650,6.6147,78.21,38.0,0.0,4.61,DZ,1.660790e+09,
223,223,Termez,37.2242,67.2783,77.09,38.0,0.0,0.00,UZ,1.660790e+09,
235,235,Astoria,46.1879,-123.8313,75.09,60.0,0.0,9.22,US,1.660790e+09,
259,259,Gaza,31.5000,34.4667,72.88,74.0,0.0,1.70,PS,1.660790e+09,
341,341,Takoradze,4.8845,-1.7554,72.55,86.0,0.0,6.31,GH,1.660791e+09,
357,357,Golden,39.7555,-105.2211,73.89,46.0,0.0,1.99,US,1.660790e+09,
422,422,Lincoln,40.8000,-96.6670,75.11,68.0,0.0,3.44,US,1.660790e+09,


In [9]:
# create a loop through dataframe
params={
    "radius": 5000,
    "type": "lodging",
    "key":g_key
    }

for index, row in hotel_df.iterrows():
    # get longitude and latitude for each row
    params['location'] = f"{row['Lat']}, {row['Lng']}"
    
    baseURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response=requests.get(baseURL,params=params).json()
    
    try:
        # populate the array with the hotel name
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]["name"]
    except: 
        print(f" Error finding lodging in {row['City']}. Skipping...")
        pass
hotel_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
123,123,Yangi Marg`ilon,40.4272,71.7189,75.92,35.0,0.0,4.61,UZ,1.660790e+09,Ikathouse
145,145,Ürümqi,43.8010,87.6005,70.75,27.0,0.0,2.24,CN,1.660790e+09,Hua Ling Grand Hotel Urumqi
158,158,San Javier,37.8063,-0.8374,76.98,74.0,0.0,4.00,ES,1.660790e+09,Hotel Mar Menor
162,162,Constantine,36.3650,6.6147,78.21,38.0,0.0,4.61,DZ,1.660790e+09,Grand Hotel Cirta
223,223,Termez,37.2242,67.2783,77.09,38.0,0.0,0.00,UZ,1.660790e+09,Hotel Sharq
235,235,Astoria,46.1879,-123.8313,75.09,60.0,0.0,9.22,US,1.660790e+09,Best Western Astoria Bayfront Hotel
259,259,Gaza,31.5000,34.4667,72.88,74.0,0.0,1.70,PS,1.660790e+09,رابطة الفنانين الفلسطينيين
341,341,Takoradze,4.8845,-1.7554,72.55,86.0,0.0,6.31,GH,1.660791e+09,Raybow International Hotel
357,357,Golden,39.7555,-105.2211,73.89,46.0,0.0,1.99,US,1.660790e+09,"The Golden Hotel, Ascend Hotel Collection"
422,422,Lincoln,40.8000,-96.6670,75.11,68.0,0.0,3.44,US,1.660790e+09,Courtyard by Marriott Lincoln Downtown/Haymarket


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))